In [1]:
import numpy as np
import pandas as pd
from scipy.spatial import distance
from sklearn.datasets import load_iris
from functools import partial
from random import random
from sklearn.preprocessing import MinMaxScaler
from collections import defaultdict
from collections import Counter

In [2]:
df = load_iris()
df.data.shape

(150, 4)

In [3]:
def f_hash(w,r,b,x):
    return int((np.dot(w,x)+b)/r)

In [8]:
class KNNHash(object):
    def __init__(self,m,L,nn):
        self.m = m
        self.L = L
        self.nn = nn

    def fit(self,X,y):
        self.t_hh = [] #hash table
        for j in range(self.L):
            f_hh = [] #compositional hash function
            for i in range(self.m):
                w = np.random.rand(1,X[0].shape[0]) #  weights of a hash function
                f_hh.append(partial(f_hash,w = w,r=random(),b=random())) # list of initialized hash function
            self.t_hh.append(
                (defaultdict(list),f_hh)
            )
        for n in range(X.shape[0]): 
            for j in range(self.L):
                ind = 0
                for i in range(self.m):
                    ind = ind + self.t_hh[j][1][i](x=X[n]) #calculation of index in hash table, simply sum of all hash func
                self.t_hh[j][0][ind].append((X[n],y[n])) #saving sample into corresponding index
    
    def predict(self,u):
        dist = 0
        euclid_distances=[]
        for j in range(self.L):
            inds = []
            for i in range(self.m):
                inds.append(self.t_hh[j][1][i](x=u))
            cntr = Counter([outp for inpt,outp in self.t_hh[j][0][sum(inds)]])
            print(cntr)
            for k in range(len(self.t_hh[j][0][sum(inds)])):
                euclid_distances.append(distance.euclidean(u,self.t_hh[j][0][sum(inds)][k][0]))
        dist = min(euclid_distances)
        print(dist)#closest distance
            
            #Here you must put your code, extend the method with distance function and calculation with unknown sample "u"

In [9]:
scaler = MinMaxScaler()
scaler.fit(df.data)
x = scaler.transform(df.data)
y = df.target

In [10]:
scaler = MinMaxScaler()
scaler.fit(df.data)
x = scaler.transform(df.data)
y = df.target
knnhash = KNNHash(4,4,4)
test1x = x[0]
test2x = x[75]
test3x = x[149]
test1y = y[0]
test2y = y[75]
test3y = y[149]
x = np.delete(x,[0,75,149],axis=0)
y = np.delete(y,[0,75,149],axis=0)

knnhash.fit(x,y)
print(test1y)
knnhash.predict(test1x)
print("-------------")
print(test2y)
knnhash.predict(test2x)
print("-------------")
print(test3y)
knnhash.predict(test3x)

0
Counter({0: 14})
Counter({0: 17, 1: 4, 2: 1})
Counter({0: 15, 1: 2})
Counter({0: 2, 1: 1})
0.03254041656427031
-------------
1
Counter({1: 9, 2: 5})
Counter({1: 15, 2: 13})
Counter({2: 9, 1: 1})
Counter({1: 2})
0.05007710104811091
-------------
2
Counter({1: 9, 2: 5})
Counter({1: 15, 2: 13})
Counter({1: 7, 2: 5})
Counter({2: 2})
0.057940218203017
